Windwerte und Niederschlag historisch und aktuell aus https://www.dwd.de/DE/leistungen/klimadatendeutschland/klarchivstunden.html

* stundenwerte_FF_02014_akt.zip (Wind, Hannover, aktuell)
* stundenwerte_FF_02014_hist.zip (Wind, Hannover, historisch)
* stundenwerte_RR_02014_akt.zip (Niederschlag, Hannover, aktuell)
* stundenwerte_RR_02014_hist.zip (Niederschlag, Hannover, historisch)

In [2]:
import pandas as pd
from dotenv import load_dotenv

from wetter import load_raw_dataframes,get_wind_data,get_rain_data


load_dotenv()
 
raw_dataframes = load_raw_dataframes()

raw_dataframes['FF']['hist'].tail() # wind historic data sample

,STATIONS_ID,MESS_DATUM,QN_3,F,D,eor
639312,2014,2022123119,3,6.1,220,eor
639313,2014,2022123120,3,7.8,210,eor
639314,2014,2022123121,3,8.0,220,eor
639315,2014,2022123122,3,7.7,220,eor
639316,2014,2022123123,3,8.9,230,eor


In [3]:
import numpy as np

FLUGBAHN_DIR = 30 # 03 & 21

timeframe_start,timeframe_end = 2021010100,2021123123

df_wind = get_wind_data(timeframe_start,timeframe_end ,13)
df_rain = get_rain_data(timeframe_start,timeframe_end , 13)

df_wind = df_wind.join(df_rain.set_index("MESS_DATUM"), on="MESS_DATUM", validate="1:1" )

#print(df_wind.columns)
# 
df_wind['wind_alignment'] = df_wind.apply(lambda row: np.sin((row['direction'] - FLUGBAHN_DIR )* np.pi / 180.)**2 , axis=1)

df_wind.head()



,MESS_DATUM,strength,direction,percip,wind_alignment
621941,2021010113,3.4,260,0.0,0.586824
621965,2021010213,1.4,230,0.0,0.116978
621989,2021010313,6.2,30,0.2,0.000000
622013,2021010413,4.2,50,0.3,0.116978
622037,2021010513,3.7,50,0.0,0.116978
